## Storing Things

In [ ]:
my_data = {'project': 'insurance_payments',
           'payment_values': [1000, 1500, 1200]}

In [ ]:
with open('tmp_file', 'w') as data_file:
    data_file.write(str(my_data))

In [ ]:
recovered_data = {}
with open('tmp_file') as data_file:
    recovered_data = eval(data_file.read())
recovered_data

# Don't try this at home

What if your file contains:

     import subprocess
     subprocess.call(['rm', '-rf', '~'])

Eval will mess you up!

# More limitations

In [ ]:
class DummyClass(object):
    def __init__(self, n):
        self.n = n

In [ ]:
my_data = [DummyClass(i) for i in range(10)]

In [ ]:
eval(str(my_data))

# Pickle

In [ ]:
import pickle

In [ ]:
with open('tmp_file', 'w') as data_file:
    pickle.dump(my_data, data_file)

In [ ]:
recovered_data = []
with open('tmp_file') as data_file:
    recovered_data = pickle.load(data_file)
recovered_data

# Databases

Databases are places to permanently store data.

## Installation

### Mac

    brew update
    brew install mongodb
    brew services start mongodb

### Linux

    apt install mongodb

or

    yum install mongodb

https://docs.mongodb.com/manual/reference/operator/query/

In [ ]:
from pymongo import MongoClient
from datetime import datetime

In [ ]:
MongoClient('localhost', 27017)

In [ ]:
db = MongoClient().musicians.info

In [ ]:
jackson = {'name': {'first': 'Michael',
                    'last': 'Jackson',
                    'middle': 'Joseph'},
           'born': datetime(1958, 8, 29),
           'died': datetime(2009, 6, 25),
           'albums': [{'name': 'Thriller',
                       'released': 1982},
                      {'name': 'Bad',
                       'released': 1987}]}

In [ ]:
collection.drop()

In [ ]:
result = db.insert_one(jackson)

In [ ]:
result.acknowledged

In [ ]:
result.inserted_id

In [ ]:
db.find_one(result.inserted_id)

In [ ]:
db.find_one()

In [ ]:
db.find()

In [ ]:
list(db.find().limit(10))

In [ ]:
db.find_one({'born': datetime(1958,8,29)})

In [ ]:
db.find_one({'name.first': 'Michael'})

In [ ]:
db.find_one({'albums.released': 1982})

In [ ]:
db.find_one({'albums.released': {'$gt': 2000}})

In [ ]:
db.find_one({'albums.released': {'$gt': 1985}})

In [ ]:
# AND
db.find_one({'name.first': 'Michael', 'name.last': 'Bowie'})

In [ ]:
db.find_one({'$or': [{'name.first': 'Michael'},
                             {'name.last': 'Bowie'}]})

In [ ]:
bowie = {'name': {'first': 'David',
                  'middle': 'Robert',
                  'last': 'Bowie'},
         'born': datetime(1947, 1, 8)}

In [ ]:
db.insert_one(bowie)

In [ ]:
list(db.find({}, {'name.last': 1, 'born': 1}))

In [ ]:
list(db.find({}, {'name.last': 1, 'born': 1}).sort('born'))

In [ ]:
from pymongo import DESCENDING

In [ ]:
list(db.find({}, {'name.last': 1, 'born': 1})\
     .sort([('born', DESCENDING)]))

## Modifying things

In [ ]:
result = db.update_one({'name.last': 'Bowie'},
                       {'$set': {'albums': []}})

In [ ]:
result.matched_count

In [ ]:
result.modified_count

In [ ]:
result = db.update_one({'name.last': 'Bowie'},
                       {'$push': {'albums': {'name': "Let's Dance",
                                             'released': 1983}}})

In [ ]:
result.raw_result

In [ ]:
db.find_one({'name.last': 'Bowie'})

In [ ]:
db.update_one({'name.last': 'Bowie'}, {'$inc': {'albums.0.released': 1}})

In [ ]:
db.find_one({'name.last': 'Bowie'}, {'albums': 1})

In [ ]:
db.update_one({'name.last': 'Bowie','albums.name': "Let's Dance"},
              {'$inc': {'albums.$.released': -1}})

In [ ]:
db.find_one({'name.last': 'Bowie'}, {'albums': 1})

# Aggregations

In [ ]:
agg = db.aggregate([{'$group': {'_id': '$born',
                                'count': {'$sum': 1}}}])

In [ ]:
list(agg)

In [ ]:
list(db.aggregate([{'$group': {'_id': {'$year': '$born'},
                               'count': {'$sum': 1}}}]))

In [ ]:
list(db.aggregate([{'$group': {'_id': {'$year': '$born'}, 
                               'max_records_released' :
                                 {'$max': {'$size': '$albums'}}}}]))

In [ ]:
list(db.aggregate([{'$match': {'name.first': 'David'}},
                   {'$group': {'_id': {'$year': '$born'}, 
                               'max_records_released' : 
                                 {'$max': {'$size': '$albums'}}}}]))

In [ ]:
list(db.aggregate([{'$project': {'last_record': {'$max': '$albums.released'}}}]))

In [ ]:
from bson import Code

In [ ]:
map_fn = Code("""function() {emit(this.born.getFullYear(), this.albums.length)}""")
reduce_fn = Code('function(key, values) {return Array.sum(values)} ')

In [ ]:
result = db.map_reduce(map_fn, reduce_fn, 'result')

In [ ]:
list(result.find())